# Testing Different Elements of the M2M API

Using elements of the M2M API to explore and understand how it all works

In [1]:
import datetime
import glob
import json
import netrc
import os
import pickle
import re
import requests
import time

import numpy as np
import pandas as pd
import xarray as xr

from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm

In [2]:
nc = netrc.netrc()
auth = nc.authenticators('ooinet.oceanobservatories.org')
BASE_URL = 'https://ooinet.oceanobservatories.org/api/m2m/'  # base M2M URL
DEPLOY_URL = '12587/events/deployment/inv/'                  # Deployment Information
SENSOR_URL = '12576/sensor/inv/'                             # Sensor Information
ANNO_URL = '12580/anno/'                                     # Annotations Information

In [18]:
# Create a list of the nodes for this site
SITE = 'CE01ISSM/'
r = requests.get(BASE_URL + DEPLOY_URL + SITE, auth=(auth[0], auth[2]))
data = r.json()
print(json.dumps(data, indent=2))

[
  "MFC31",
  "MFD35",
  "MFD37",
  "RID16",
  "SBC11",
  "SBD17"
]


In [19]:
# Determine list of sensors for each node
NODE = 'RID16/'
r = requests.get(BASE_URL + DEPLOY_URL + SITE + NODE, auth=(auth[0], auth[2]))
data = r.json()
print(json.dumps(data, indent=2))

[
  "00-DCLENG000",
  "01-OPTAAD000",
  "02-FLORTD000",
  "03-CTDBPC000",
  "03-DOSTAD000",
  "04-VELPTA000",
  "05-PCO2WB000",
  "06-PHSEND000",
  "07-NUTNRB000",
  "08-SPKIRB000"
]


In [20]:
# Create a list of deployments for this sensor 
SENSOR = '03-CTDBPC000/'
r = requests.get(BASE_URL + DEPLOY_URL + SITE + NODE + SENSOR, auth=(auth[0], auth[2]))
data = r.json()
print(json.dumps(data, indent=2))

[
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11
]


In [21]:
# Determine the date and time range for the deployment
DEPLOY = '11'
r = requests.get(BASE_URL + DEPLOY_URL + SITE + NODE + SENSOR + DEPLOY, auth=(auth[0], auth[2]))
data = r.json()
START = time.strftime('%Y-%m-%dT%H:%M:%S.000Z', time.gmtime(data[0]['eventStartTime']/1000.))
if data[0]['eventStopTime']:
    STOP = time.strftime('%Y-%m-%dT%H:%M:%S.000Z', time.gmtime(data[0]['eventStoptTime']/1000.))
else:
    STOP = time.strftime('%Y-%m-%dT%H:%M:%S.000Z', time.gmtime(time.time()))


In [22]:
# Determine the delivery methods available for this sensor
r = requests.get(BASE_URL + SENSOR_URL + SITE + NODE + SENSOR, auth=(auth[0], auth[2]))
data = r.json()
print(json.dumps(data, indent=2))

[
  "recovered_host",
  "recovered_inst",
  "telemetered"
]


In [23]:
# Determine the streams associated with the delivery method available for this sensor
METHOD = 'telemetered/'
r = requests.get(BASE_URL + SENSOR_URL + SITE + NODE + SENSOR + METHOD, auth=(auth[0], auth[2]))
data = r.json()
print(json.dumps(data, indent=2))

[
  "ctdbp_cdef_dcl_instrument"
]


In [25]:
# Grab the provenance info and the driver associated with this data stream for this deployment
STREAM = 'ctdbp_cdef_dcl_instrument'
OPTIONS = '?beginDT=' + START + '&endDT=' + STOP + '&limit=2&parameters=7&include_provenance=true&strict_range=true'
r = requests.get(BASE_URL + SENSOR_URL + SITE + NODE + SENSOR + METHOD + STREAM + OPTIONS, auth=(auth[0], auth[2]))
data = r.json()
print(json.dumps(data, indent=2))

# note this will fail if the system does not have any data, we want to know that so will need to code for failures

{
  "data": [
    {
      "pk": {
        "node": "RID16",
        "stream": "ctdbp_cdef_dcl_instrument",
        "subsite": "CE01ISSM",
        "deployment": 11,
        "time": 3764719869.483,
        "sensor": "03-CTDBPC000",
        "method": "telemetered"
      },
      "provenance": "ba0d3067-bc6b-48e6-b986-a85eb883477e",
      "time": 3764719869.483
    },
    {
      "pk": {
        "node": "RID16",
        "stream": "ctdbp_cdef_dcl_instrument",
        "subsite": "CE01ISSM",
        "deployment": 11,
        "time": 3771743411.115,
        "sensor": "03-CTDBPC000",
        "method": "telemetered"
      },
      "provenance": "8aa65616-505c-4cd2-b4b0-1afea43dafbf",
      "time": 3771743411.115
    }
  ],
  "provenance": {
    "8aa65616-505c-4cd2-b4b0-1afea43dafbf": {
      "file_name": "/omc_data/whoi/OMC/CE01ISSM/D00011/cg_data/dcl16/ctdbp1/20190710.ctdbp1.log",
      "parser_name": "mi.dataset.driver.ctdbp_cdef.dcl.ctdbp_cdef_dcl_telemetered_driver",
      "parser_version": "

}


In [26]:
# determine the driver used to parse the data and the source of the data
provenance = data['provenance']
driver = [provenance[key]['parser_name'] for key in provenance.keys()][0].encode('ascii', 'ignore')
infile = [provenance[key]['file_name'] for key in provenance.keys()][0].encode('ascii', 'ignore')

In [30]:
# now request the same data from OOINet via the M2M system
OPTIONS = '?beginDT=' + START + '&endDT=' + STOP + '&format=application/netcdf'
r = requests.get(BASE_URL + SENSOR_URL + SITE + NODE + SENSOR + METHOD + STREAM + OPTIONS, auth=(auth[0], auth[2]))
data = r.json()

# wait until the request is completed
check_complete = data['allURLs'][1] + '/status.txt'
with tqdm(total=1200, desc='Processing Request:') as pbar:
    for i in range(1200):
        r = requests.get(check_complete)
        pbar.update(1)
        if r.status_code == requests.codes.ok:
            pbar.n = 1200
            pbar.last_print_n = 1200
            pbar.refresh()
            print('request completed in %f minutes' % elapsed)
            break
        else:
            time.sleep(.5)
            elapsed = (i * 0.5) / 60


request completed in 0.891667 minutes


In [31]:
# Function to create a list of the NetCDF data files created in the request above
def list_files(url, tag=''):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    pattern = re.compile(str(tag))
    return [node.get('href') for node in soup.find_all('a', text=pattern)]

# Function to download one of the NetCDF files, convert to an xarray dataset and convert to time as 
# the appropriate dimension instead of obs...whatever that is.
def process_file(file):
    # download and convert the data
    baseurl = 'https://opendap.oceanobservatories.org/thredds/dodsC/'
    url = re.sub('catalog.html\?dataset=', baseurl, file)
    ds = xr.open_dataset(url).load()  # download the data first, before converting
    ds = ds.swap_dims({'obs': 'time'})
    ds = ds.drop(['obs', 'driver_timestamp', 'ingestion_timestamp', 'port_timestamp', 'preferred_timestamp'])
    
    return ds

In [32]:
# Create a list of the files from the request above using a simple regex as tag to discriminate the files
files = list_files(data['allURLs'][0], '.*CTDBP.*\.nc$')

# Process the data files for and concatenate into a single dataframe
frames = [process_file(f) for f in files]
m2m = xr.concat(frames, 'time')

In [33]:
m2m

<xarray.Dataset>
Dimensions:                         (time: 983)
Coordinates:
  * time                            (time) datetime64[ns] 2019-04-20T03:31:09.482999808 ... 2019-07-10T10:30:11.114999808
    pressure                        (time) float32 7.328 7.163 ... 7.108 7.273
    lat                             (time) float64 44.66 44.66 ... 44.66 44.66
    lon                             (time) float64 -124.1 -124.1 ... -124.1
Data variables:
    deployment                      (time) int32 11 11 11 11 11 ... 11 11 11 11
    id                              (time) |S64 b'ee28742a-a4a0-4fcc-a0b0-7d67b519154e' ... b'1d30fd6a-62d8-4d56-945d-62a389dff62e'
    conductivity                    (time) float32 3.51582 3.53364 ... 3.86881
    date_time_string                (time) object b'empty' b'empty' ... b'empty'
    dcl_controller_timestamp        (time) object b'empty' b'empty' ... b'empty'
    internal_timestamp              (time) datetime64[ns] 2019-04-20T03:31:03 ... 2019-07-10T10:3